In [ ]:
from ccdproc import ImageFileCollection

In [ ]:
from pathlib import Path

In [ ]:
object_name = "OB0006"

In [ ]:
DATADIR = Path(f"../imaging/{object_name}")

In [ ]:
def get_paths(dirname, data_dir=DATADIR):
    """Given the image superdirectory, returns the subdirectories corresponding to the 
    bias, flat, standard, and object images respectively
    """
    bias_dir = data_dir.joinpath(dirname, "bias")
    flat_dir = data_dir.joinpath(dirname, "flat")
    std_dir = data_dir.joinpath(dirname, "stds")
    obj_dir = data_dir.joinpath(dirname, "object")

    return bias_dir, flat_dir, std_dir, obj_dir

In [ ]:
allimgs_order = ["bias", "flat", "std", "obj"]

In [ ]:
def get_image_files(dirname, data_dir):
    """Given an image superdirectory, retrieve ImageFileCollections
    for each of the subdirectories of images: bias, flat, stds, and object
    """
    paths = get_paths(dirname, data_dir)
    return [ImageFileCollection(path) for path in paths]

In [ ]:
for ob in range(1, 10):
    print(f"OB000{ob}")
    print("-"*100)
    print()
    allimgs = get_image_files("", Path(f"raw/OB000{ob}"))
    for (imgs, name) in zip(allimgs, allimgs_order):
        print(name)
        imgs.summary[["file", "filter2", "ra", "dec", "exptime", "date"]].pprint(max_width=120)
        print()

    print("\n\n")

# Create catalogue human names

In [ ]:
allimgs_order

In [ ]:
allnames_bias = {
    "20230816": [],
    "20230723": [],
    "20230709": [],
    "20230708": [],
}

allnames_flat = {
    "20230811": {},
    "20230723": {},
    "20230709": {},
    "20230708": {},
}

allnames_std = {
    "20230816": {},
    "20230723": {},
    "20230709": {},
    "20230708": {},
}

allnames_obj = {
    "yasone3_20230816": {},
    "yasone3_20230723": {},
    "yasone2_20230709": {},
    "yasone1_20230708": {},
}

for ob in range(1, 10):
    
    imgs_bias, imgs_flat, imgs_std, imgs_obj = get_image_files("", Path(f"raw/OB000{ob}"))

    date = imgs_bias.summary["file"][0].split("-")[1]
    repeated_calibration = len(allnames_bias[date]) > 0

    ob_calibration = {1:1, 2:1, 3:1, 4:4, 5:4, 6:4, 7:7, 8:8, 9:8}[ob]
    dirname = f"OB000{ob}/"
    dirname_calibration = f"OB000{ob_calibration}/"

    for row in imgs_bias.summary:
        filename = str(row["file"])
        date = filename.split("-")[1]
        if repeated_calibration:
            assert dirname_calibration + "bias/" + filename in allnames_bias[date]
        else:
            allnames_bias[date].append(dirname + "bias/" + filename)


    for row in imgs_flat.summary:
        filename = str(row["file"])
        date = filename.split("-")[1]
        filt = row["filter2"].split("Sloan_")[1]
        
        if repeated_calibration:
            assert dirname_calibration + "flat/" + filename in allnames_flat[date][filt]
        else:
            if filt not in allnames_flat[date].keys():
                allnames_flat[date][filt] = []
                
            allnames_flat[date][filt].append(dirname + "flat/" + filename)


    for row in imgs_std.summary:
        filename = str(row["file"])
        date = filename.split("-")[1]
        filt = row["filter2"].split("Sloan_")[1]
        
        if repeated_calibration:
            assert dirname_calibration + "stds/" + filename in allnames_std[date][filt]
        else:
            if filt not in allnames_std[date].keys():
                allnames_std[date][filt] = []
                
            allnames_std[date][filt].append(dirname + "stds/" + filename)


    for row in imgs_obj.summary:
        filename = str(row["file"])
        date = filename.split("-")[1]
        yasone_num = row["object"].split("Candidate")[1]
        obj_date = f"yasone{yasone_num}_{date}"
        filt = row["filter2"].split("Sloan_")[1]
        
        if filt not in allnames_obj[obj_date].keys():
            allnames_obj[obj_date][filt] = []

        allnames_obj[obj_date][filt].append(dirname + "object/" + filename)

In [ ]:
allnames_flat

In [ ]:
allnames_bias

In [ ]:
allnames_std

In [ ]:
print(allnames_obj)

allnames_obj["yasone1"] = allnames_obj.pop("yasone1_20230708")
allnames_obj["yasone2"] = allnames_obj.pop("yasone2_20230709")
allnames_obj["yasone3a"] = allnames_obj.pop("yasone3_20230723")
allnames_obj["yasone3b"] = allnames_obj.pop("yasone3_20230816")


In [ ]:
allnames_obj.keys()

In [ ]:
def to_filename_dict_nofilt(allnames, basename="bias"):
    d_out = {}
    for group, d in allnames.items():
        d_out[group] = {}
        i = 0
        for filename in d:
            i += 1
            newname =  f"{basename}_{i:02d}.fits"
            d_out[group][newname] = filename

    return d_out

In [ ]:
def to_filename_dict(allnames, basename):
    d_out = {}
    for group, d in allnames.items():
        d_out[group] = {}
        for key, items in d.items():
            i = 0
            for filename in items:
                i += 1
                newname =  f"{basename}_{key}_{i:02d}.fits"
                d_out[group][newname] = filename

    return d_out

In [ ]:
to_filename_dict(allnames_flat, "flat")

In [ ]:
to_filename_dict(allnames_std, "std")

In [ ]:
to_filename_dict(allnames_obj, "img")

In [ ]:
import tomli_w

In [ ]:
bias_dict = to_filename_dict_nofilt(allnames_bias, "bias")
flat_dict = to_filename_dict(allnames_flat, "flat")

flat_dict["20230816"] = flat_dict.pop("20230811")

std_dict = to_filename_dict(allnames_std, "std")
obj_dict = to_filename_dict(allnames_obj, "obj")

In [ ]:
calibration_dict = bias_dict.copy()
for key in bias_dict.keys():
    calibration_dict[key] |= flat_dict[key]
    calibration_dict[key] |= std_dict[key]

In [ ]:
calibration_dict

In [ ]:
allfilenames = calibration_dict | obj_dict

In [ ]:
allfilenames

In [ ]:
with open("image_keys.toml", "wb") as f:
    tomli_w.dump(allfilenames, f)